In [850]:
# https://www.kaggle.com/dimasuwandi/car-price-from-mobil123com-may-2020

from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as uReq
import numpy as np

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [857]:
my_url = 'https://www.mobil123.com/mobil-dijual/indonesia?type=used&page_number=1&page_size=25'
uClient = uReq(my_url)
page_html = uClient.read()
uClient.close()
page_soup = soup(page_html, 'html.parser')
    
containers = page_soup.findAll('div', {'class': 'grid'})
container = containers[0]
            

In [725]:
raw = container.findAll('article', {'class':'listing listing--card box relative push--top js--listing js--multi-lead'})
lst = []
ls = []

for i in raw:
    lst.append(i.text.replace('\n\n\n\n\n','').replace('                      ','').replace('    ',' ').replace('\n\n','\n').replace('  ',' ').replace('\n','|').replace('||','|').replace('| ','|').replace(' |','|').replace('||','|'))


In [858]:
lst[10]

'|13|Bekas|Premium|21285 KM|Automatic|DKI Jakarta|Sales Agent|Dealer terpercaya memiliki rekam jejak yang terbukti menegakkan praktik jual mobil terbaik disertifikasi oleh Mobil123.com|Kontak|WhatsApp|Bandingkan|Simpan 2018 Honda CR-V 1.5 Prestige Prestige VTEC SUV|Harga khusus credit 385 jt - km on going 21 rban - tangan 1 dr baru - nopol ganjil - ...|Rp 380.000.000|Kontak|WhatsApp|Bandingkan|Simpan|'

# Done

In [1035]:
# filename='dataScraping/mobil123.csv'
# f = open(filename,'a')
# headers='condition,mileage,transmition,seller,year,brand,body,model,engV,types,price\n'
# f.write(headers)
# f.close()

pg = 4078

while(True):
    
    my_url = f'https://www.mobil123.com/mobil-dijual/indonesia?type=used&page_number={pg}&page_size=25'
    uClient = uReq(my_url)
    page_html = uClient.read()
    uClient.close()
    page_soup = soup(page_html, 'html.parser')

    containers = page_soup.findAll('div', {'class': 'grid'})
    container = containers[0]
    
    pg += 1


    raw = container.findAll('article', {'class':'listing listing--card box relative push--top js--listing js--multi-lead'})
    lst = []
    ls = []
    
    

    if len(raw) > 0:
        for i in raw:
            lst.append(i.text.replace('\n\n\n\n\n','').replace('                      ','').replace('    ',' ').replace('\n\n','\n').replace('  ',' ').replace('\n','|').replace('||','|').replace('| ','|').replace(' |','|').replace('||','|'))

        for k in range(len(lst)):
            lstBreak = lst[k].split('|')

            for i in range(len(lstBreak)):
                if lstBreak[i] in ['Dealer terpercaya memiliki rekam jejak yang terbukti menegakkan praktik jual mobil terbaik disertifikasi oleh Mobil123.com','Kontak','WhatsApp','Bandingkan']:
                    lstBreak[i] = 'removed'

                elif lstBreak[i] == 'Simpan':
                    lstBreak[i] = 'removed'




            condition = ''
            mileage = ''
            transmition = ''
            seller = ''
            year = ''
            brand = ''
            model = ''
            engV = ''
            types = ''
            body = ''
            price = ''
            for j in lstBreak:
                for i in range(len(lstBreak)):
                    lst1 = lstBreak[i]
                    lst1FirstBreak = lst1.split(' ')
                    lst1Break = lst1.split(' ')

                    if ('Simpan') in ' '.join(lst1FirstBreak):
                        break1 = ' '.join(lst1FirstBreak).split(' - ')
                        lst1Break = break1[0].split(' ')
                        idxEngV = 0
                        year = lst1Break[1]
                        brand = lst1Break[2]
                        
                        if '.' not in lst1Break[-4]:
                            if lst1Break[-4] != brand  and len(lst1Break[-4]) > 1:
                                model = lst1Break[-4]
                            else:
                                model = lst1Break[3]
                        else:
                            model = lst1Break[3]                                
                        
                        for i in lst1Break:
                            if len(i) == 3 and '.' in i:
                                engV = i     
                        
                                
                        if '.' not in lst1Break[-2]:
                            types = lst1Break[-2]
                        elif '.' not in lst1Break[4]:
                            types = lst1Break[4]   
                        else:
                            types = lst1Break[-1]                           

                        if len(lst1Break[-1]) > 1:
                            body = lst1Break[-1]
                        else :
                            body = lst1FirstBreak[-1]
                    
                        
                    elif ('KM') in ' '.join(lst1Break):

                        if ('-') in ' '.join(lst1Break):
                            ml = lstBreak[i].replace('-','').split(' ')
                            ml1 = ml[0]
                            ml2 = ml[2]
                            
                            
                            if len(ml1) > 3 or len(ml2) > 3:
                                if len(ml1) > 3:
                                    ml1 = ml1[0:-3]
                                else :
                                    ml1 = ml1


                                if len(ml2) > 3:
                                    if 'K' in ml2 or 'k' in ml2:
                                        ml2 = ml2[0:-4] + 'k'
                                    else:
                                        ml2 = ml2[0:-3] + 'k'
                                        
                                mileage = str(ml1+' - '+ml2).replace('K','k') + ' KM'
                                    
                            
                            else :
                                mls2 = lst1Break[2].replace('K','k')
                                mile = lst1Break[0] + ' - ' + lst1Break[2] + ' KM'
                                mileage = mile

                        
                        elif ('-') not in ' '.join(lst1Break):
                            ml = lstBreak[i].split(' ')
                            ml1 = ml[0]

                            if len(ml1) > 3:
                                ml1 = ml1[0:-3]
                            else :
                                continue
                                
                            ml2 = str(int(ml1) + 5) + 'k'
                                
                            mileage = str(ml1+' - '+ml2).replace('K','k') + ' KM'
                            
                        else:
                            mls2 = lst1Break[2].replace('K','k')
                            mile = lst1Break[0] + ' - ' + lst1Break[2] + ' KM'
                            mileage = mile

                    elif lst1Break[0] in ['Bekas','Baru']:
                        condition = lst1Break[0]

                    elif lst1Break[0] in ['Automatic','Manual']:
                        transmition = lst1Break[0]

                    elif 'Sales' in ' '.join(lst1Break) or 'Agent' in ' '.join(lst1Break) or 'Dealer' in ' '.join(lst1Break):
                        seller = ' '.join(lst1Break)

                    elif ('Rp ') in ' '.join(lst1Break):
                        price = lst1Break[1].replace('.','')  

            
            
            filename='dataScraping/mobil123.csv'
            f = open(filename,'a')
            f.write(condition+','+mileage+','+transmition+','+seller+','+str(year)+','+brand+','+body+','+model+','+str(engV)+','+types+','+str(price)+'\n')
            f.close()
            
#             print('kondisi:',condition,', mileage:',mileage,
#                   ', tranmisiton:',transmition,', seller:',seller,
#                   ', year:',year,', brand:',brand,', body:',body,
#                   ', model:',model,', engV:',engV,', types:',types,
#                   ', price:',price,'\n')

    else:
        print('DONE!!')
        break
        
    print('page',pg)  
    
#     if pg == 10:
#         break



    #     print(condition,mileage,
    #           transmition,seller,
    #           year,brand,body,
    #           model,engV,types,
    #           price,'\n')

page 4079
page 4080
page 4081
page 4082
page 4083
page 4084
page 4085
page 4086
page 4087
page 4088
page 4089
page 4090
page 4091
page 4092
page 4093
page 4094
page 4095
page 4096
page 4097
page 4098
page 4099
page 4100
page 4101
page 4102
page 4103
page 4104
page 4105
page 4106
page 4107
page 4108
page 4109
page 4110
page 4111
page 4112
page 4113
page 4114
page 4115
page 4116
page 4117
page 4118
page 4119
page 4120
page 4121
page 4122
page 4123
page 4124
page 4125
page 4126
page 4127
page 4128
page 4129
page 4130
page 4131
page 4132
page 4133
page 4134
page 4135
page 4136
page 4137
page 4138
page 4139
page 4140
page 4141
page 4142
page 4143
page 4144
page 4145
page 4146
page 4147
page 4148
page 4149
page 4150
page 4151
page 4152
page 4153
page 4154
page 4155
DONE!!


In [1037]:
import pandas as pd
df = pd.read_csv('dataScraping/mobil123.csv')
len(df)
df

,condition,mileage,transmition,seller,year,brand,body,model,engV,types,price
0,Bekas,230 - k KM,Automatic,Dealer,2012,Honda,SUV,CR-V,2.4,SUV,200000000
1,Bekas,40 - 45k KM,Automatic,Dealer,2007,Mercedes-Benz,Sedan,S500,5.5,L,420000000
2,Bekas,25 - 30k KM,Automatic,Dealer,2011,Land,SUV,SDV6,3.0,Autobiography,790000000
3,Bekas,30 - 35k KM,Automatic,Dealer,2012,Land,SUV,SDV6,3.0,Autobiography,795000000
4,Bekas,36 - 41k KM,Automatic,Dealer,2014,Land,SUV,Rover,5.0,Dynamic,1380000000
5,Bekas,21 - 26k KM,Automatic,Dealer,2013,Mercedes-Benz,Hatchback,B200,1.6,Urban,225000000
6,Bekas,60 - 65k KM,Manual,Dealer,2016,Toyota,MPV,Innova,2.4,G,270000000
7,Bekas,10 - 15k KM,Automatic,Dealer,2016,Mercedes-Benz,Sedan,S400,3.0,Exclusive,1435000000
8,Bekas,36 - 41k KM,Automatic,Dealer,2006,Mercedes-Benz,Hatchback,A150,1.5,Hatchback,98000000
9,Bekas,36 - 41k KM,Automatic,Dealer,2006,Mercedes-Benz,Hatchback,A150,1.5,Hatchback,98000000


In [1038]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103230 entries, 0 to 103229
Data columns (total 11 columns):
condition      103230 non-null object
mileage        102655 non-null object
transmition    103230 non-null object
seller         96756 non-null object
year           103230 non-null object
brand          103230 non-null object
body           103229 non-null object
model          103230 non-null object
engV           103216 non-null object
types          103228 non-null object
price          103230 non-null object
dtypes: object(11)
memory usage: 8.7+ MB


In [1039]:
df.describe()

,condition,mileage,transmition,seller,year,brand,body,model,engV,types,price
count,103230,102655,103230,96756,103230,103230,103229,103230,103216,103228,103230
unique,3,346,3,3,109,65,75,691,117,665,4043
top,Bekas,40 - 45k KM,Automatic,Dealer,2016,Toyota,MPV,Avanza,1.5,G,115000000
freq,103202,5947,70390,73615,8858,32120,32037,5598,19626,11723,1116
